In [ ]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import contextily
import mapclassify
import folium
import aiohttp
import fsspec


# Load the GeoDataFrame containing geometry shapes for US counties
county_data_url = "https://github.com/babdelfa/gis/blob/main/counties_geometry.zip?raw=true"
with fsspec.open(county_data_url) as file:
    county_shapes = gpd.read_file(file)

# Load the DataFrame containing COVID-19 data
covid_data_url = "https://github.com/CSSEGISandData/COVID-19/raw/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv"
df = pd.read_csv(covid_data_url)

# Merge the GeoDataFrame and DataFrame based on the FIPS code
gdf = county_shapes.merge(df, left_on='FIPS_BEA', right_on='FIPS')

# Get user input for the county
county_name = input("Enter County: ").strip()  # Remove leading and trailing whitespaces

# Filter the data for the specified county
selected_county = gdf[gdf['Admin2'].str.strip().str.lower() == county_name.lower()]

# Check if the county exists in the data
if not selected_county.empty:
    # Display population information
    population_info = selected_county['Population'].values[0] if 'Population' in selected_county.columns else "Population information not available"

    # Display the first reported outbreak date
    first_outbreak_info = selected_county['First_Outbreak'].values[0] if 'First_Outbreak' in selected_county.columns else "First Reported Outbreak information not available"

    # Display COVID-19 Summary Statistics
    print("\n*** MIS 433 COVID19 Report ***")
    print(f"Enter County: {county_name}")
    print(f"Population of {county_name}, Virginia: {population_info}")
    print(f"First Reported Outbreak in {county_name}: {first_outbreak_info}")

    # Define a set of columns for COVID-19 statistics
    covid_columns = ['average_new_cases_2020', 'average_new_cases_2021', 'average_new_cases_2022',
                     'total_new_cases_2020', 'total_new_cases_2021', 'total_new_cases_2022',
                     'cumulative_total_cases_2022']

    # Check if the necessary columns are available for COVID-19 statistics
    if all(col in selected_county.columns for col in covid_columns):
        # Display COVID-19 statistics with actual values
        print(f"{county_name} County COVID19 Summary Statistics:")
        print(f" - Average number of new cases in 2020: {selected_county['average_new_cases_2020'].values[0]:.2f}")
        print(f" - Average number of new cases in 2021: {selected_county['average_new_cases_2021'].values[0]:.2f}")
        print(f" - Average number of new cases in 2022: {selected_county['average_new_cases_2022'].values[0]:.2f}")
        print(f" - Total number of new cases in 2020: {selected_county['total_new_cases_2020'].values[0]:,}")
        print(f" - Total number of new cases in 2021: {selected_county['total_new_cases_2021'].values[0]:,}")
        print(f" - Total number of new cases in 2022: {selected_county['total_new_cases_2022'].values[0]:,}")
        print(f" - Cumulative total number of cases: {selected_county['cumulative_total_cases_2022'].values[0]:,} (December 31, 2022)")
    else:
        print(f"COVID-19 statistics not available for {county_name}")
else:
    print(f"County '{county_name}' not found in the data.")
